The general optimization problem is \\
$$\text{min} \ \  \sum_{i=1}^{6} |P_i-\bar{P_i}| \\
\text{s.t.  } \\
\bar{P_i}=b_0+b_1L_i+b_2E_i \ \ \forall i  \\
b_0 \geq 0$$ \\

\\
No! the optimization problem is not linear because the objective function involves absolut value function, which is not linear.

**using approach 1** \\

Our objective is to minimize $|R_i| \ i.e. |P_i-\bar{P_i}| \ \ \forall i=1 \ to \ 6$ \\

Here we will use the result that |x| can be written as Min z s.t. z >= x and z >= -x \\

Hence the optimization problem becomes \\
$$\text{min} \ \  \sum_{i=1}^{6} z_i \\
\text{s.t.  } \\
z_i \geq P_i-\bar{P_i} \ \ \forall i \\
z_i \geq \bar{P_i}-P_i \ \ \forall i\\
\bar{P_i}=b_0+b_1L_i+b_2E_i \ \ \forall i  \\
b_0 \geq 0$$ \\

Which can be re written as follows: \\
$$\text{min} \ \  \sum_{i=1}^{6} z_i \\
\text{s.t.  } \\
z_i +b_0+b_1L_i+b_2E_i\geq  P_i \ \ \ \ \ \forall i \\
z_i -b_0-b_1L_i-b_2E_i \geq - P_i \ \ \forall i \\
b_0 \geq 0 \\
$$


In [ ]:
!pip install -q pyomo

In [ ]:
from pyomo.environ import *

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
data_csvfile = pd.read_csv('lab6_ex3.csv')

In [ ]:
data_csvfile.columns

Index(['warehouse', 'seling price', 'lot size', 'elevation'], dtype='object')

In [ ]:
data_csvfile['warehouse']

0    1
1    2
2    3
3    4
4    5
5    6
Name: warehouse, dtype: int64

In [ ]:
data_csvfile.index

RangeIndex(start=0, stop=6, step=1)

In [ ]:
len(data_csvfile.index)

6

In [ ]:
# create a model
model_lab6_ex3 = ConcreteModel()

In [ ]:
# here N denotes no. of variables
N = len(data_csvfile.index)
n = 3 

In [ ]:
M = 2 #no of constraints

In [ ]:
col_indices = np.arange(N)
b_indices = np.arange(n)
row_indices = np.arange(M)
row_indices

array([0, 1])

In [ ]:
#declaring the decision variables in the model
model_lab6_ex3.z = Var(col_indices)
model_lab6_ex3.b = Var(b_indices)

In [ ]:
#objection function
model_lab6_ex3.objective = Objective(expr=summation(model_lab6_ex3.z),sense=minimize)

In [ ]:
model_lab6_ex3.constraints = ConstraintList()

In [ ]:
for i in col_indices:
  model_lab6_ex3.constraints.add(model_lab6_ex3.z[i]+model_lab6_ex3.b[0]+model_lab6_ex3.b[1]*data_csvfile['lot size'][i]+model_lab6_ex3.b[2]*data_csvfile['elevation'][i] >= data_csvfile['seling price'][i])

In [ ]:
for i in col_indices:
  model_lab6_ex3.constraints.add(model_lab6_ex3.z[i]-model_lab6_ex3.b[0]-model_lab6_ex3.b[1]*data_csvfile['lot size'][i]-model_lab6_ex3.b[2]*data_csvfile['elevation'][i] >= -data_csvfile['seling price'][i])

In [ ]:
model_lab6_ex3.b[0].setlb(0)

In [ ]:
model_lab6_ex3.pprint()

3 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    z_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

2 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
    z : Size=6, Index=z_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  Tru

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model_lab6_ex3)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
# display solution
print('\nOptimal sum of residuals = ', model_lab6_ex3.objective())

print('\nNew Decision Variables')
for i in col_indices:
  print('z[',i+1,']=', model_lab6_ex3.z[i].value)

print('\nOriginal Decision Variables')
for i in b_indices:
  print('b[',i,']=', model_lab6_ex3.b[i].value)
for i in col_indices:
  print(u'P\u0304[',i,']=', model_lab6_ex3.b[0].value+model_lab6_ex3.b[1].value*data_csvfile['lot size'][i] +model_lab6_ex3.b[2].value*data_csvfile['elevation'][i] )

print('\nConstraints')
model_lab6_ex3.constraints.display()


Optimal sum of residuals =  125454.54499999993

New Decision Variables
z[ 1 ]= 38818.182
z[ 2 ]= -2.7009345e-11
z[ 3 ]= 54272.727
z[ 4 ]= 32363.636
z[ 5 ]= -2.7785522e-11
z[ 6 ]= -1.671118e-11

Original Decision Variables
b[ 0 ]= 104181.82
b[ 1 ]= 67.272727
b[ 2 ]= -356.36364
P̄[ 0 ]= 136181.81860000006
P̄[ 1 ]= 149999.9996
P̄[ 2 ]= 179272.72650000005
P̄[ 3 ]= 122363.63760000002
P̄[ 4 ]= 80000.00090000003
P̄[ 5 ]= 120000.00150000001

Constraints
constraints : Size=12
    Key : Lower     : Body                : Upper
      1 :  175000.0 :  175000.00060000003 :  None
      2 :  150000.0 :         149999.9996 :  None
      3 :  125000.0 :  233545.45350000006 :  None
      4 :   90000.0 :  154727.27360000001 :  None
      5 :   80000.0 :          80000.0009 :  None
      6 :  120000.0 :  120000.00149999998 :  None
      7 : -175000.0 :  -97363.63660000001 :  None
      8 : -150000.0 : -149999.99960000007 :  None
      9 : -125000.0 : -124999.99950000003 :  None
     10 :  -90000.0 :  -900

**Approach 2** \\
$Let \ \ P_i-\bar{P_i}=u_i-v_i, \ \ and \ \ |P_i-\bar{P_i}|=u_i+v_i ,u_i \geq 0, v_i \geq 0 \ \ \ \forall i=1,2,...6 \\
$ \\
The constraint $$\bar{P_i}=b_0+b_1L_i+b_2E_i $$ can be written as \\
$$P_i-\bar{P_i}=P_i-b_0-b_1L_i-b_2E_i$$ \\
i.e again implies $$P_i-\bar{P_i}+b_0+b_1L_i+b_2E_i=P_i$$
Then the optimization problem becomes \\
$$\text{Min  } \sum_{i=1}^{6} (u_i+v_i) \\
\text{s.t.} \\
u_i-v_i+b_0+b_1L_i+b_2E_i=P_i \ \ \forall i \\
u_i \geq 0, v_i \geq 0 \ \ \forall i\\
b_0 \geq 0
$$

In [ ]:
model2_lab6_ex3 = ConcreteModel()

In [ ]:
#declaring the decision variables in the model
model2_lab6_ex3.u = Var(col_indices)
model2_lab6_ex3.v = Var(col_indices)

model2_lab6_ex3.b = Var(b_indices)

In [ ]:
#objection function
model2_lab6_ex3.objective = Objective(expr=summation(model2_lab6_ex3.u)+summation(model2_lab6_ex3.v),sense=minimize)

In [ ]:
model2_lab6_ex3.constraints = ConstraintList()

In [ ]:
data_csvfile['lot size'][2]

3500

In [ ]:
for i in col_indices:
  model2_lab6_ex3.constraints.add(model2_lab6_ex3.u[i]-model2_lab6_ex3.v[i]+model2_lab6_ex3.b[0]+model2_lab6_ex3.b[1]*data_csvfile['lot size'][i]+model2_lab6_ex3.b[2]*data_csvfile['elevation'][i] == data_csvfile['seling price'][i])

In [ ]:
for i in col_indices:
  model2_lab6_ex3.u[i].setlb(0)
  model2_lab6_ex3.v[i].setlb(0)

In [ ]:
model2_lab6_ex3.pprint()

4 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    u_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    v_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

3 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
    u : Size=6, Index=u_index
        Key : Lower : Value : Upper : Fi

In [ ]:
result2 = opt_cbc.solve(model2_lab6_ex3)
print('Solver status:', result2.solver.status)
print('Solver termination condition:',result2.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
# display solution
print('\nOptimal sum of residuals = ', model2_lab6_ex3.objective())

print('\nNew Decision Variables')
for i in col_indices:
  print('u[',i+1,']=', model2_lab6_ex3.u[i].value)
for i in col_indices:
  print('v[',i+1,']=', model2_lab6_ex3.v[i].value)

print('\nOriginal Decision Variables')
for i in b_indices:
  print('b[',i,']=', model2_lab6_ex3.b[i].value)
for i in col_indices:
  print(u'P\u0304[',i,']=', model_lab6_ex3.b[0].value+model_lab6_ex3.b[1].value*data_csvfile['lot size'][i] +model_lab6_ex3.b[2].value*data_csvfile['elevation'][i] )

print('\nConstraints')
model2_lab6_ex3.constraints.display()


Optimal sum of residuals =  125454.545

New Decision Variables
u[ 1 ]= 38818.182
u[ 2 ]= 0.0
u[ 3 ]= 0.0
u[ 4 ]= 0.0
u[ 5 ]= 0.0
u[ 6 ]= 0.0
v[ 1 ]= 0.0
v[ 2 ]= 0.0
v[ 3 ]= 54272.727
v[ 4 ]= 32363.636
v[ 5 ]= 0.0
v[ 6 ]= 0.0

Original Decision Variables
b[ 0 ]= 104181.82
b[ 1 ]= 67.272727
b[ 2 ]= -356.36364
P̄[ 0 ]= 136181.81860000006
P̄[ 1 ]= 149999.9996
P̄[ 2 ]= 179272.72650000005
P̄[ 3 ]= 122363.63760000002
P̄[ 4 ]= 80000.00090000003
P̄[ 5 ]= 120000.00150000001

Constraints
constraints : Size=6
    Key : Lower    : Body               : Upper
      1 : 175000.0 : 175000.00060000003 : 175000.0
      2 : 150000.0 :        149999.9996 : 150000.0
      3 : 125000.0 : 124999.99950000003 : 125000.0
      4 :  90000.0 :  90000.00160000002 :  90000.0
      5 :  80000.0 :  80000.00090000003 :  80000.0
      6 : 120000.0 : 120000.00150000001 : 120000.0


In both cases we are getting the same optimal value and solutions.